In [4]:
# SNV: single nucleotide variants
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
home_dir = "../../"

import pandas as pd
from utils.pandas_extented_filters import filter_multi_base_variants, separate_ref_and_alt_allele

In [14]:
inp_filepath = home_dir+"data/clinvar/filtered/clinvar_HumanPathogenicMissenseVariants01012022To14022023.txt"
# inp_filepath = home_dir+"data/clinvar/filtered/clinvar_HumanLikelyPathogenicMissenseVariants01012022To14022023.txt"
col_names = ["GRCh38Chromosome", "GRCh38Location", "Canonical SPDI"]
df = pd.read_csv(inp_filepath, delim_whitespace=False, sep="\t")
print(df.shape)
# print(df.columns)
# df.head()

(3916, 34)


In [15]:
chromosomal_variants_df = df[col_names]
temp = chromosomal_variants_df["Canonical SPDI"].apply(separate_ref_and_alt_allele) # filter 
chromosomal_variants_df["ref"], chromosomal_variants_df["alt"] = temp.apply(lambda x: x[0]), temp.apply(lambda x: x[1])

after_multibase_removal_df = chromosomal_variants_df[chromosomal_variants_df["ref"].apply(filter_multi_base_variants)] # filter: removing multi-nucleotide base variants 
after_multibase_removal_df = after_multibase_removal_df[after_multibase_removal_df["alt"].apply(filter_multi_base_variants)] # filter: removing multi-nucleotide base variants 
after_multibase_removal_df["GRCh38Chromosome"] = after_multibase_removal_df["GRCh38Chromosome"].apply(lambda x: x.split("|")[0])
after_multibase_removal_df

/tmp/ipykernel_3519000/1605079769.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chromosomal_variants_df["ref"], chromosomal_variants_df["alt"] = temp.apply(lambda x: x[0]), temp.apply(lambda x: x[1])
/tmp/ipykernel_3519000/1605079769.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chromosomal_variants_df["ref"], chromosomal_variants_df["alt"] = temp.apply(lambda x: x[0]), temp.apply(lambda x: x[1])


,GRCh38Chromosome,GRCh38Location,Canonical SPDI,ref,alt
0,1,976215,NC_000001.11:976214:A:G,A,G
1,1,1232280,NC_000001.11:1232279:T:C,T,C
2,1,1232281,NC_000001.11:1232280:G:A,G,A
3,1,1516036,NC_000001.11:1516035:T:G,T,G
4,1,1527718,NC_000001.11:1527717:A:G,A,G
...,...,...,...,...,...
3911,X,154957013,NC_000023.11:154957012:G:A,G,A
3912,X,154966097,NC_000023.11:154966096:C:A,C,A
3913,X,154966433,NC_000023.11:154966432:C:G,C,G
3914,X,154966667,NC_000023.11:154966666:T:G,T,G


In [16]:
out_filepath = home_dir+"models/aa_common/inputs/" + inp_filepath.split("/")[-1]
after_multibase_removal_df[["GRCh38Chromosome", "GRCh38Location", "ref", "alt"]].to_csv(out_filepath, index=False, sep=" ", header=False)